# Исследование надёжности заёмщиков

Входные данные - статистика о платёжеспособности клиентов банка.

Цель исследования - определить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

В ходе исследования нужно получить ответы на следующие вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучаем датасет

In [91]:
# Устанавливаем библиотеку для лемматизации текста
!pip install pymystem3

#Загружаем библиотеки
import pandas as pd

from pymystem3 import Mystem
m = Mystem()

from collections import Counter

from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

In [92]:
# загружаем данные
df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [93]:
# Функция для предварительного ознакомления с данными. 

def hello_data(data):
    
    ''' Выводит первые строки, 
                info, 
                уникальные значения
                и пропуски, если они есть '''
    
    columns = data.columns 
    display(data.head(10))
    display(data.info())
    lost_values = data.duplicated().sum()
   
    print(
        f'Явных дубликатов в данных: {lost_values}. Это {lost_values / len(data) * 100:.2f}% от всех значений.'
         )
    print()
        
    for column in columns: # для каждой колонки
        try:
            unique_value = (len(data[column].unique())) # считаем уникальные значения 
        except AssertionError:
            pass
        
        if data[column].isna().sum() != 0: # если в колонке есть пропуски
            print( f'Уникальных значений в колонке {column} - {unique_value}') #выводим количество уникальных значений
            persent = (data[column].isna().sum() / len(data[column]) * 100) # считаем сколько это в процентах и округляем методом "round"
            print('Пропущенных строк в колонке {} - {:.1f} %'.format(column, persent))
            print("_"*20)
        else: print( f'Уникальных значений в колонке {column} - {unique_value}') # если пропусков не нашли, то выводим количество уникальных значений  
     
hello_data(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Явных дубликатов в данных: 54. Это 0.25% от всех значений.

Уникальных значений в колонке children - 8
Уникальных значений в колонке days_employed - 19352
Пропущенных строк в колонке days_employed - 10.1 %
____________________
Уникальных значений в колонке dob_years - 58
Уникальных значений в колонке education - 15
Уникальных значений в колонке education_id - 5
Уникальных значений в колонке family_status - 5
Уникальных значений в колонке family_status_id - 5
Уникальных значений в колонке gender - 3
Уникальных значений в колонке income_type - 8
Уникальных значений в колонке debt - 2
Уникальных значений в колонке total_income - 19352
Пропущенных строк в колонке total_income - 10.1 %
____________________
Уникальных значений в колонке purpose - 38


**Вывод**

Полученный датасет содержит информацию, хранящуюся в следующих колонках:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита
    
При предварительном ознакомлении с данными выявили следующие проблемы:
1. Есть незначительное количество дублирующихся данных - 0.25%.
2. Есть по 10,1 % пропущенных значений в колонках "days_employed" и "total_income". 
3. Колонка "education" содержит значения в разном регистре,что приводит к появлению неявных дубликатов. Сейчас в ней 15 уровней образования.
4. В "purpose" цели получения кредита по смыслу можно объединить в категории, но записаны они в свободной форме, к примеру, "сыграть свадьбу" и "на проведение свадьбы". Эти значения необходимо будет категоризовать.

## Предобработка данных

### Обработка пропусков

In [94]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [95]:
# По двум колонкам ищем пропуски и находим сумму строк для которых обе колонки содержат Nan,
# сравниваем ее с количеством пропусков одной из колонок "days_employed"

if df['days_employed'].isna().sum() == df[['days_employed', 'total_income']].isna().all(axis=1).sum():
    print("Пропуски полностью совпадают.")
else:
    print("Пропуски не совпадают полностью.")

Пропуски полностью совпадают.


Предположение подтвердилось. Отсутствующий трудовой стаж и отсутствующий ежемесячный доход связаны между собой. Возможно, пропуски относятся к одной категории заемщиков, например, "безработный" или "студент". Проверим это, подсчитаем количество пропущеных значений по категориям заемщиков.

In [96]:
# методами size и count ищем разницу по сгруппированным даным. 
# size суммирует количество всех значений в колонке, count игнорирует при подсчете Nan
x = display(
    df.groupby('income_type')['days_employed'].size() - df.groupby('income_type')['days_employed'].count()
)

income_type
безработный           0
в декрете             0
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
студент               0
Name: days_employed, dtype: int64

Пропущенные значения нельзя отнести к одной группе.  На данный момент всем неизвестным значениям 
в таблице присвоено значение NaN. Чтобы это не повлияло  на точность расчетов значения в колонке 
отражающей доход - "total_income" заменим на медианное значение в соответствии с группой заемщиков.


In [97]:
# Добавляем столбец, в котором будут храниться средние значения.
df['median_income'] = df.groupby('income_type')['total_income'].transform('median')
# Заполняем пропущенные значения средними.
df['total_income'] = df['total_income'].fillna(df['median_income'])
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,172357.950966
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,172357.950966
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,142594.396847
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,142594.396847
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,142594.396847


**Вывод**

   В результате предварительного исследования данных был обнаружен пропуск в 2174 строках в столбцах:
"days_employed" и "total_income" — общий трудовой стаж в днях и ежемесячный доход. Это пропуски взаимосвязаны и  неслучайны, но отнести их к одной группе заемщиков нельзя. Можно объяснить тем, что часть клиентов банка работает неофициально, либо в силу других обстоятельств банк не затребовал с этих клиентов данные о стаже и заработке. 

   Отдельно поговорим о стаже - days_employed. Отрицательные значения и тип данных 'float' наводят на мысль, что данные сохраняются или переданы некорректно. Стоит обсудить эту находку с дата-инженерами. В текущем исследовании эти данные не понадобятся и были оставлены в первоначальном виде. 

### Замена типа данных  

Чтобы не было так мучительно больно смотреть на колонку 'total_income', округлим до целых значений. Заменим тип данных float  на int методом 'astype', как более универсальный и простой. В данном исследовании точность до копеек нам не нужна.

In [98]:
df['total_income'] = df['total_income'].astype('int')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594.396847
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412


### Обработка дубликатов и поиск аномалий в данных

Ранее мы обнаружили незначительное количество дубликатов. Удаляем их.

In [99]:
# удаляем явные дубликаты, обновляем индексы строк
df = df.drop_duplicates().reset_index(drop=True) 

Исследуем данные на предмет наличия неявных дубликатов и возможных выбросов. 

Найдем уникальные значения для колонок, с подсчетом количества. Нас интересуют колонки:

- education — уровень образования клиента
- family_status — семейное положение
- dob_years — возраст клиента в годах
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- purpose — цель получения кредита

При предварительном ознакомлении с данными было выявленно, что колонка 'education' содержит данные
в разном регистре. 

In [100]:
# Приведем в единый формат столбец 'education' понижением регистра.
df['education'] = df['education'].str.lower()
# и посмотрим сколько уникальных значений получилось.
df['education'].value_counts() 

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь порядок. Получили 5 возможных уровней образования. Среди заемщиков подавляющее большинство- люди со средним образованием. Можно прдположить, что это связано с уровнем дохода, таким людям не хватает ежемесячных доходов на крупные покупки и они чаще обращаются в банк за кредитом. Малое количество заемщиков с начальным уровнем образования или неоконченным высшим можно обяснить отказами банка в получении кредита в связи с низким уровнем дохода, это могут быть молодые люди с неподтвержденным стажем работы. Более детально зависимость между образованием и получением кредита в рамках данного проекта изучать не будем, т.к. это выходит за рамки текущей задачи.

In [101]:
 # проверим семейный статус
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Немного смущают категории "женат/ замужем" и "гражданский брак". По определению это - синонимы, впрочем, в обществе широко используется определение "гражданский брак" как обозначение простого сожительства. Оставим категории как есть, не ясно как банк интерпретирует такую информацию, но предполагаем, что она нужна именно в таком виде.

В любом случае видим, что большинство заемщиков состоят в браке. Парадокс, но людям, ведущим совместное хозяйство, сильнее не хватает денег, чем одиноким. 

В датасете для колонки 'family_status' предусмотрен столбец с id статуса. Нужно проверить корректно ли записаны id.

In [102]:
# Сгруппируем статусы по колонкам 'family_status', 'family_status_id' и посмотрим какой id к чему относится:
df.groupby(['family_status', 'family_status_id'])\
  .size()\
  .to_frame()

,,0
family_status,family_status_id,
Не женат / не замужем,4,2810
в разводе,3,1195
вдовец / вдова,2,959
гражданский брак,1,4163
женат / замужем,0,12344


Определили 5 статусов. Соответственно:
- 0 - женат / замужем
- 1 - гражданский брак
- 2 - вдовец / вдова
- 3 - в разводе
- 4 - Не женат / не замужем    

Теперь займемся категорией дети. Проверим значения столбца количества детей на аномалии.

In [103]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Получили выбросы по количеству детей в виде значений -1 ребенок(47 строк) и 20 детей(76 строк). Так как мы не знаем как возникла ошибка и количество таких значений мало, то заменим их медианным значением. Удалять строки не будем, чтобы сохранить другую информацию из этих строк.

In [104]:
# Меняем аномальные значения в колонке 'children' на медианное
children_median = df.query('children != 20 & children != -1')['children'].median() 
df['children'] = df['children'].replace([-1,20], children_median)
# проверяем корректность замены
df['children'].value_counts().to_frame()

,children
0.0,14230
1.0,4809
2.0,2052
3.0,330
4.0,41
5.0,9


In [105]:
# Теперь посмотрим на возраст. Выведем уникальные значения возраста с сортировкой по возрастанию
df.sort_values('dob_years')['dob_years'].unique() 

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

In [106]:
# считаем сколько строк с  0 возрастом
len(df.query('dob_years == 0'))

101

В 101 строке выявили возраст равный 0, остальные значения выглядят правдоподобными. Заменим 0 возраст на медианный и перейдем к категории "тип занятости".

In [107]:
# Заменим медианным значением:
age_median = df.query('dob_years == 0')['dob_years'].median() # выбираем значения не равные 0 и считаем медиану.
df['dob_years'] = df['dob_years'].replace(0, age_median)

# проверяем категорию "тип занятости"
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Все в порядке. Поищем аномамалии по целям кредита:

In [108]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [109]:
len(df['purpose'].unique()) # нашли количество уникальных значений

38

Имеем 38 очень похожих вариантов цели кредитования. Судя по всему в кредитной анкете поле "цель кредита" заполняется в свободной форме, а стоило бы предусмотреть выбор из нескольких возможных вариантов. Будем делить цели кредита на категории с помощью лемматизации.

### Лемматизация

In [110]:
# основные ключевые слова можно выделить вручную
categories = ["жилье", "образование", "свадьба", "недвижимость", "автомобиль"]

# напишем функцию, которая преобразовывает фразу в лемму, сравнивает ее со списком значений 'categories'
# и если слово из 'categories' есть в лемме меняет лемму на это слово.
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma

In [111]:
# добавляем к датасету колонку 'purpose_group' и заполняем ее значениями из 'categories'
df['purpose_group'] = df['purpose'].apply(lemmatize)       

In [112]:
# Считаем все возможные категории целей кредита.
display(df['purpose_group'].value_counts())
df['purpose_group'].shape

недвижимость    6353
жилье           4461
автомобиль      4308
образование     4014
свадьба         2335
Name: purpose_group, dtype: int64

(21471,)

In [114]:
#Теперь можно объединить "недвижимость" и "жилье" в общую группу
df['purpose_group'] =df['purpose_group'].replace('жилье', 'недвижимость')
 # проверка
display(df['purpose_group'].value_counts())
df['purpose_group'].shape

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose_group, dtype: int64

(21471,)

**Вывод**

В ходе предобработки данных решили 4 найденых ранее проблемы в данных. Убрали явные дубликаты, привели в соответствие уровни образования. Изучили пропуски в колонках "days_employed" и "total_income". Выянилось, что пропуски в данных не случайны и относятся к одним и тем же строкам. Предположение, что данные отсутствуют у молодых людей, которые еще не работали и не имеют стажа и заработка не подтвердилось. Очевидно у части клиентов банк не затребовал данные. Возможно, в некоторых случаях (сейчас или ранее) это было необязательное поле анкеты. Из 38 уникальных значений целей кредита с помощью лемматизации сократили список до 4. Рекомендовано в будущем доработать анкету на получение кредита и предусмотреть выбор только из нескольких возможных вариантов.

## Категоризация данных

Мы уже категоризовали цели получения кредита с помощью лемматизации. Для дальнейшего исследования сгруппируем наши данные по уровню доходов и колличеству детей. 

Воспользуемся методом qcut, чтобы в каждом интервале содержалось равное количество наблюдений.

In [71]:
# делим на 4 равные категории
pd.qcut(df['total_income'], 4)

0        (195767.5, 2265604.0]
1         (107654.5, 142594.0]
2         (142594.0, 195767.5]
3        (195767.5, 2265604.0]
4         (142594.0, 195767.5]
                 ...          
21466    (195767.5, 2265604.0]
21467     (142594.0, 195767.5]
21468    (20666.999, 107654.5]
21469    (195767.5, 2265604.0]
21470    (20666.999, 107654.5]
Name: total_income, Length: 21471, dtype: category
Categories (4, interval[float64]): [(20666.999, 107654.5] < (107654.5, 142594.0] < (142594.0, 195767.5] < (195767.5, 2265604.0]]

Получили 4 равные категории по уровню доходов и разделим по группам так:

- **"низкий доход"** - доход до 107 тысяч 620 рублей,
- **"средний доход"** - от 107 тысяч 621 рублей до 142 тысяч 594 рублей,
- **"доход выше среднего"** - от 142 тысяч 595 рублей до 195 тысяч 818 рублей,
- **"высокий доход"** - от 195 тысяч 819 рублей и больше.
Напишем для этой цели функцию:

In [57]:
# функция для деления по уровню доходов
def incom_category(row):
    if row['total_income'] <= 107620:
        return "низкий доход"
    elif 107620 < row['total_income'] <= 142594:
        return "средний доход"
    elif 142594 < row['total_income'] <= 195818:
        return "доход выше среднего"
    else:
        return "высокий доход"

In [58]:
# добавляем столбец с категориями по доходу в датафрейм
df['incom_category'] = df.apply(incom_category , axis=1) 
# проверяем
df['incom_category'].unique()

array(['высокий доход', 'средний доход', 'доход выше среднего',
       'низкий доход'], dtype=object)

По количеству детей интересно будет сравнить результаты по принципам дети есть/ детей нет, а также в зависимости от количества детей.
Для начала разобьем данные на 3 категории:

- "нет детей"
- " 1-2 ребенка"
- "многодетные"

In [118]:
# функция для разделения на группы по количеству детей 
def children_category(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

In [119]:
# добавляем столбец в датафрейм
df['children_category'] = df.apply(children_category, axis=1) 

In [120]:
# функция для разделения по принципу дети есть(1) / детей нет (0)
def have_kids(row):
    if row['children'] == 0:
        return 0
    else:
        return 1

In [121]:
# добавляем столбец в датафрейм
df['having_kids'] = df.apply(have_kids, axis=1) 
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,purpose_group,having_kids,children_category
0,1.0,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594.396847,недвижимость,1,1-2 ребенка
1,1.0,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847,автомобиль,1,1-2 ребенка
2,0.0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594.396847,недвижимость,0,нет детей
3,3.0,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594.396847,образование,1,многодетные
4,0.0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412,свадьба,0,нет детей


**Вывод**

Провели категоризацию по уровню доходов и количеству детей, заполнили соответствующими данными датафрейм. Теперь можно переходить к анализу полученных данных и ответам на поставленные вопросы.

## Исследование зависимостей
### Зависимость между наличием детей и возвратом кредита в срок

Построим сводную таблицу по категориям есть дети(1) / нет детей(0) и возвратом кредита в срок, где

    debt = 0 - это отсутствие просрочек по кредитам,
    debt = 1 -  это наличие долгов:

In [123]:
df_pivot_kids = df.pivot_table(
                index=['having_kids'], 
                columns='debt', 
                values = 'children', 
                aggfunc='count'
                ).

df_pivot_kids

debt,0,1
having_kids,,
0,13158,1072
1,6572,669


Получили значения по задолженности для бездетных - 1072 и 669 для тех, у кого дети есть. Корректнее будет посчитать отношение должников не в общем, а внутри группы.

In [124]:
# посчитаем по группам отношение общего количества значений в группе к количеству должников в группе
# и добавим в колонку 'ratio'
df_pivot_kids ['ratio'] = df_pivot_kids [1] / (df_pivot_kids [0] + df_pivot_kids [1])*100
# сортируем 'ratio'по убыванию
df_pivot_kids.sort_values('ratio', ascending = False)

debt,0,1,ratio
having_kids,,,
1,6572,669,9.239055
0,13158,1072,7.533380


Наличие детей повышает риск невозврата. Теперь посмотрим более детально на зависимость просрочек от количества детей. Построим сводную таблицу:

<div class="alert alert-success">

<b>🔁1️⃣ Ревьюер: 👍</b> Верно.

</div>

In [ ]:
df_pivot_children = df.pivot_table(
    index=['children_category'], 
    columns='debt', 
    values = 'children', 
    aggfunc='count'
)

In [ ]:
# посчитаем по группам отношение общего количества значений в группе к количеству должников в группе
# и добавим в колонку 'ratio'
df_pivot_children ['ratio'] = (df_pivot_children [1] /
                              (df_pivot_children [0] + 
                               df_pivot_children [1]) * 100)
# сортируем 'ratio'по убыванию
df_pivot_children.sort_values('ratio', ascending = False)

**Вывод**

Да, возврат клиентом кредита в срок действительно имеет зависимость от наличия детей. Среди бездетных самый низкий процент тех,кто не отдает кредит - 7,53 %. Но при этом среди тех, у кого дети есть, дисциплинированней многодетные 8,5 %, против 9,2% у семей с 1-2 детьми.





<div class="alert alert-success">

<b>🔁1️⃣ Ревьюер: 👍</b> Многодетные нерепрезентативная выборка, ее лучше не учитывать.

</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

<div class="alert alert-info">

<b>Студент: </b> Закоментила часть про зависимость по принципу есть пара/ нет пары.

</div>

<div class="alert alert-success">

<b>🔁2️⃣ Ревьюер: 👍</b> Ok.

</div>

In [ ]:
#df_pivot_family = df.pivot_table(
    #index=['family_category'], 
    #columns='debt', 
    #values = 'family_status', 
    #aggfunc='count')
#df_pivot_family ['ratio'] = df_pivot_family [1] / (df_pivot_family [0] + df_pivot_family [1]) * 100
# сортируем 'ratio'по убыванию
#df_pivot_family.sort_values('ratio', ascending = False)

Сведем в таблицу данные по семейным статусам и наличию долгов. Ранее получили обозначение статусов:
- 0 - женат / замужем
- 1 - гражданский брак
- 2 - вдовец / вдова
- 3 - в разводе
- 4 - Не женат / не замужем    

In [ ]:
df_pivot_family = df.pivot_table(
    index=['family_status_id'], 
    columns='debt', 
    values = 'family_status', 
    aggfunc='count'
)
df_pivot_family

In [ ]:
# посчитаем по группам отношение общего количества значений в группе к количеству должников в группе и добавим в колонку 'ratio'
df_pivot_family ['ratio'] = df_pivot_family [1] / (df_pivot_family [0] + df_pivot_family [1]) * 100
# сортируем 'ratio'по убыванию
df_pivot_family.sort_values('ratio', ascending = False)

**Вывод**

При изучении данных по семейному положению больше всего должников выявилось среди людей со статусом "не женат" - 9,75%, на втором месте "Гражданский брак" - 9,32 %. Вдовцы(6,56%) оказались лучшими клиентами чем женатые (7,54%). В целом видно как с возрастом, проходя разные жизненные этапы, люди более ответственно подходят к обслуживанию кредитов.


<div class="alert alert-warning">

<b>🔁1️⃣ Ревьюер: 👉</b> Попробуй разделить эти статусы не по наличию партнера, а по возрасту и все станет на свои места. Сама посуди, если человек никогда не был в браке и вдовец – это разные люди? Скорее всего первый молодой, а второй в возрасте, соответственно и подход к выплате долгов у них будет разный.

</div>

<div class="alert alert-info">

<b>Студент: </b> Скорректировала вывод с учетом категории "гражданский брак". Действительно так нагляднее получилось.

</div>

<div class="alert alert-success">

<b>🔁2️⃣ Ревьюер: 👍</b> Все правильно.

</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Делаем сводную таблицу по доходу:

In [ ]:
df_pivot_total_income = df.pivot_table(
    index=['incom_category'], 
    columns='debt', 
    values = 'family_status', 
    aggfunc='count'
)
# и сразу добавим в колонку 'ratio'
df_pivot_total_income ['ratio'] = df_pivot_total_income [1] / (df_pivot_total_income [0] + df_pivot_total_income [1]) * 100
# сортируем 'ratio'по убыванию
df_pivot_total_income.sort_values('ratio', ascending = False)

**Вывод**

Неожиданно, но люди со средним и выше среднего доходом имеют больше невыплаченных кредитов (8,85% и 8,71%), чем люди с низким доходом(8.04%). Лучше всех дела обстоят у людей с высоким доходом (6,95%). Зависимость и правда есть.

<div class="alert alert-success">

<b>🔁1️⃣ Ревьюер: 👍</b> Молодец.

</div>

- Как разные цели кредита влияют на его возврат в срок?

Делаем сводную таблицу по колонке цель кредита:

In [ ]:
df_pivot_purpose_group = df.pivot_table(
    index=['purpose_group'], 
    columns='debt', 
    values = 'family_status', 
    aggfunc='count'
)
# и сразу добавим в колонку 'ratio'
df_pivot_purpose_group ['ratio'] = df_pivot_purpose_group [1] / (df_pivot_purpose_group [0] + df_pivot_purpose_group [1]) * 100
# сортируем 'ratio'по убыванию
df_pivot_purpose_group.sort_values('ratio', ascending = False)

<div class="alert alert-info">

<b>Студент: </b> Скорректировала вывод с учетом категории "гражданский брак".

</div>

<div class="alert alert-success">

<b>🔁2️⃣ Ревьюер: 👍</b> Спасибо.

</div>

**Вывод**

Цифры говорят сами за себя, зависимость от цели четко прослеживается. Самый высокий процент долгов по кредитам в категории "автомобиль". Это может быть связано с тем, что люди плохо просчитывают во сколько им ежемесячно будет обходиться содержание автомобиля и берут в кредит суммы большие чем могут отдать.
На втором месте образование. Удовольствие не дешевое, а между тем, помимо обучения, нужно еще успевать зарабатывать на погашение кредита, что удается не всем. 
 

<div class="alert alert-success">

<b>🔁1️⃣ Ревьюер: 👍</b> Согласен.

</div>

## Шаг 4. Общий вывод

В ходе исследования выявилась зависимость между наличием детей и возвратом кредита в срок. Среди бездетных самый низкий процент тех, кто не отдает кредит - 7,53 %, а в семьях, где 1-2 ребенка риск возрастает до 9.29%.
Есть зависиомсть и от цели кредита. Хуже всего обстоят дела с кредитами на автомобиль - 9.35% и образование - 9.21%. Семейный статус имеет значение при возврате кредита в срок, самые ненадежные клиенты - неженатые и состоящие в гражданском браке. Люди побывавшие или состоящие в браке ответственнее относятся к долгам.
С уровнем дохода, ожидаемо, лучше справляются богатые - 6.96%, а люди со средним доходом чаще других не возвращают кредиты - 8.85%.




<div class="alert alert-warning">

<b>🔁1️⃣ Ревьюер: 👉</b> Вывод очень краткий, но по делу, здорово. Исправь информацию по семенному статусу, но это на твое усмотрение.

</div>

<div class="alert alert-success">

<b>🔁2️⃣ Ревьюер: 👍</b> Верно.

</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div class="alert alert-danger">

<b>🔁1️⃣ Ревьюер: ✋</b> Евгения, у тебя классный проект. Молодец! По существу, у меня остался один комментарий. Я хочу чтобы ты посмотрела на семейное положение не только со стороны наличия партнера, а как на этапы жизни человека.
    
Для удобства поиска я добавил тебе ссылку:
    
* [Верни категорию](#cell_13)
    
Жду новую версию проекта. 🙂

</div>

<div class="alert alert-success">

<b>🔁2️⃣ Ревьюер: ✌️</b> Евгения, спасибо за отличную работу и за интерес к новым методам. Знаю, что вам не все из них дают, но понимаю, что они вам пригодятся как в дальнейшем обучении, так и в работе. Критических замечаний нет, ты умница!
    
Поздравляю, **проект принят!** 🥳 🎉

Желаю тебе успехов в дальнейшем обучении. 😉
    
</div>